In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('../petfinder_data/favorites_sample_small.csv', delimiter=';',error_bad_lines=False)

In [ ]:
data.head(10)

In [ ]:
data.info()

In [ ]:
data['Description_Flag'] = np.where(data['pet_description'].notnull(), 1, 0)

In [ ]:
data['organization_type'].fillna("Unknown", inplace = True) 

In [ ]:
data.head(10)

In [ ]:
print("Unique Values in `Flag_Detail` => {}".format(data.Description_Flag.unique()))

In [ ]:
data_des = data[[
    'Description_Flag'
    ,'pet_description']].copy()

data_des.head()

In [ ]:
data2 = data[[
    'user_id'
    ,'id'
    ,'num_aifs'
    , 'Description_Flag'
    , 'mixed_breed']].copy()

In [ ]:
datatest = data[[
    'user_id'
    ,'id'
    ,'Description_Flag'
    ,'mixed_breed'
    ,'animal_type'
    ,'age'
    ,'state_code'
    ,'primary_breed'
    ,'organization_type'
    ,'gender'
    ,'size'
    ,'num_aifs'
    ,'house_trained'
    ,'coat_length'
    ,'num_videos'
    ,'num_photos'
    ,'good_with_dogs'
    ,'good_with_cats'
    ,'good_with_children']].copy()

In [ ]:
user_enc = LabelEncoder()
# can later use user_enc like so user_enc.inverse_transform([24230]) to get original user_id
data2['user'] = user_enc.fit_transform(data2['user_id'].values)

pet_enc = LabelEncoder()
data2['pet'] = pet_enc.fit_transform(data2['id'].values)

n_users = data2['user'].nunique()
print(n_users)
n_favorites = data2['pet'].nunique()
print(n_favorites)

In [ ]:
# validate encodings start from 0 and index to n_users-1 and n_favorites-1
print(data2.user.min())
print(data2.user.max())
print(data2.pet.min())
print(data2.pet.max())

In [ ]:
train, test = train_test_split(data2, test_size=0.2, random_state=42)

In [ ]:
print(train)

In [ ]:
fav_input = Input(shape=[1], name="Fav-Input")
fav_embedding = Embedding(n_favorites, 10, name="Fav-Embedding")(fav_input)
fav_vec = Flatten(name="Flatten-Books")(fav_embedding)


user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users, 10, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)


prod = Dot(name="Dot-Product", axes=1)([fav_vec, user_vec])
model = Model([user_input, fav_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
#Model error
#https://github.com/tensorflow/tensorflow/issues/23698

from keras.models import load_model

if os.path.exists('model.h5'):
    model = load_model('model.h5')
else:
    history = model.fit([train.user, train.pet], train.num_aifs, epochs=5, verbose=1)
    model.save('model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [ ]:
# Creating dataset for making recommendations for the first user
pet_data = np.array(list(set(train.pet)))
user = np.array([1 for i in range(len(pet_data))])
predictions = model.predict([user, pet_data])
predictions = np.array([a[0] for a in predictions])
recommended_pet_ids = (-predictions).argsort()[:5]
print(recommended_pet_ids)
print(predictions[recommended_pet_ids])